In [1]:
import xarray as xr
import pandas as pd
import numpy as np

In [2]:
# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [3]:
#Open IMERGdata
imerg2015 = xr.open_dataset('../../IMERG/IMERG_hourly_2015.nc')

smap2015 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2015_SA.nc')
smap2016 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2016_SA.nc')
smap2017 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2017_SA.nc')
smap2018 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2018_SA.nc')
smap2019 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2019_SA.nc')
smap2020 = xr.open_dataset('../../SMAP_L4_all/SMAP_L4_SMroot_concat_2020_SA.nc')

In [4]:
smap2015 = smap2015.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
smap2016 = smap2016.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
smap2017 = smap2017.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
smap2018 = smap2018.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
smap2019 = smap2019.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
smap2020 = smap2020.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)

In [5]:
smap2015  = smap2015.where((smap2015.sm>=0)&(smap2015.sm<=0.9),np.nan)
smap2016  = smap2016.where((smap2016.sm>=0)&(smap2016.sm<=0.9),np.nan)
smap2017  = smap2017.where((smap2017.sm>=0)&(smap2017.sm<=0.9),np.nan)
smap2018  = smap2018.where((smap2018.sm>=0)&(smap2018.sm<=0.9),np.nan)
smap2019  = smap2019.where((smap2019.sm>=0)&(smap2019.sm<=0.9),np.nan)
smap2020  = smap2020.where((smap2020.sm>=0)&(smap2020.sm<=0.9),np.nan)

In [6]:
merged_smap = xr.concat([smap2015,smap2016,smap2017,smap2018,smap2019,smap2020],
                         'time')

In [7]:
path='/scratch/wcq7pz/exp_levante_post/'

topo5km = xr.open_dataset(path+ 'topography_dom03_5km.nc')
topo5km.coords['lon'] = (topo5km.coords['lon'] + 180) % 360 - 180
topo5km  = topo5km.sortby(topo5km.lon)

frland=xr.open_dataset(path+'fr_land_dom03_5km.nc')
frland.coords['lon'] = (frland.coords['lon'] + 180) % 360 - 180
frland = frland.sortby(frland.lon)

In [8]:
topo10km = topo5km.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)
frland10km = frland.interp(lat = imerg2015.lat.values,lon = imerg2015.lon.values)

In [9]:
# Repeat the variable along the time dimension of merged_SMAP
ds_topo = topo10km['topography_c'].broadcast_like(merged_smap['time'])
ds_landf = frland10km['fr_land'].broadcast_like(merged_smap['time'])

In [10]:
## Remove topography <500m and non-land areas
nmerged_smap = merged_smap.where((ds_topo<=500)&(ds_landf==1),np.nan)

In [11]:
## OPEN DATAFRAMES WITH SYSTEMS' STATISTICS
#### Amazon #####
df_stage_ab_2015 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2015.pkl')
df_stage_ab_2016 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2016.pkl')
df_stage_ab_2017 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2017.pkl')
df_stage_ab_2018 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2018.pkl')
df_stage_ab_2019 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2019.pkl')
df_stage_ab_2020 = pd.read_pickle('imerg_pkl/df_imerg_stage_AB_2020.pkl')
#### SESA #####
df_stage_sesa_2015 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2015.pkl')
df_stage_sesa_2016 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2016.pkl')
df_stage_sesa_2017 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2017.pkl')
df_stage_sesa_2018 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2018.pkl')
df_stage_sesa_2019 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2019.pkl')
df_stage_sesa_2020 = pd.read_pickle('imerg_pkl/df_imerg_stage_SESA_2020.pkl')

In [12]:
### Filter times absent in SMAP 

# # Create a set of times from 'ds' for faster lookup
times_in_smap = nmerged_smap.coords['time'].values
times_in_smap_set = set(times_in_smap)

# Filter 'df' to only include rows where 'time' is in 'smap'
#### Amazon #####
df_imerg_AB_2015_filt = df_stage_ab_2015[df_stage_ab_2015['time'].isin(times_in_smap_set)]
df_imerg_AB_2016_filt = df_stage_ab_2016[df_stage_ab_2016['time'].isin(times_in_smap_set)]
df_imerg_AB_2017_filt = df_stage_ab_2017[df_stage_ab_2017['time'].isin(times_in_smap_set)]
df_imerg_AB_2018_filt = df_stage_ab_2018[df_stage_ab_2018['time'].isin(times_in_smap_set)]
df_imerg_AB_2019_filt = df_stage_ab_2019[df_stage_ab_2019['time'].isin(times_in_smap_set)]
df_imerg_AB_2020_filt = df_stage_ab_2020[df_stage_ab_2020['time'].isin(times_in_smap_set)]

#### SESA #####
df_imerg_SESA_2015_filt = df_stage_sesa_2015[df_stage_sesa_2015['time'].isin(times_in_smap_set)]
df_imerg_SESA_2016_filt = df_stage_sesa_2016[df_stage_sesa_2016['time'].isin(times_in_smap_set)]
df_imerg_SESA_2017_filt = df_stage_sesa_2017[df_stage_sesa_2017['time'].isin(times_in_smap_set)]
df_imerg_SESA_2018_filt = df_stage_sesa_2018[df_stage_sesa_2018['time'].isin(times_in_smap_set)]
df_imerg_SESA_2019_filt = df_stage_sesa_2019[df_stage_sesa_2019['time'].isin(times_in_smap_set)]
df_imerg_SESA_2020_filt = df_stage_sesa_2020[df_stage_sesa_2020['time'].isin(times_in_smap_set)]

In [13]:
print(len(df_imerg_AB_2015_filt),len(df_stage_ab_2015))

2232 10504


#### Environment definition : Select around "dist" times the radius of the systems

In [14]:
def mirrored(maxval, inc=1):
    x = np.arange(inc, maxval, inc)
    if x[-1] != maxval:
        x = np.r_[x, maxval]
    return np.r_[-x[::-1], 0, x]

In [16]:
def sel_composite(df,var1,namevar='w_so',dist=2,timelag=False,dt=1):
    """
    df = dataframe
    var1, var2 = 3D dataset of dimensions time,lat,lon for precipitation and SM, respectively.
    """    
    if timelag==True:
        
        df_cat = pd.concat([pd.DataFrame([{'time':df.time.values[i]-pd.Timedelta(dt),'local_time':df.local_time.values[i]-pd.Timedelta(dt),
                                           'feature':df.feature.values[i],
        'mean_SM_env':var1.sel(time=df.time.values[i]-pd.Timedelta(dt)).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).mean(['lat','lon'])[namevar].values, 
        'std_SM_env':var1.sel(time=df.time.values[i]-pd.Timedelta(dt)).sel(lon=slice(
        df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
        df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).std(['lat','lon'])[namevar].values}]) for i in range(len(df))],
ignore_index=False)
    else:
        df_cat = pd.concat([pd.DataFrame([{'time':df.time.values[i],'local_time':df.local_time.values[i],
                                           'feature':df.feature.values[i],
                            'mean_SM_env':var1.sel(time=df.time.values[i]).sel(lon=slice(
                            df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
                            df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).mean(['lat','lon'])[namevar].values, 
                            'std_SM_env':var1.sel(time=df.time.values[i]).sel(lon=slice(
                            df.longitude.values[i]-dist*df.radius.values[i]/110,df.longitude.values[i]+dist*df.radius.values[i]/110),lat=slice(
                            df.latitude.values[i]-dist*df.radius.values[i]/110,df.latitude.values[i]+dist*df.radius.values[i]/110)).std(['lat','lon'])[namevar].values}]) for i in range(len(df))],
              ignore_index=False) 
       
    df_cat = df_cat.reset_index(drop=True);
    df_cat['mean_SM_env'] =df_cat['mean_SM_env'].astype(float)
    df_cat['std_SM_env'] =df_cat['std_SM_env'].astype(float)
    return df_cat

#### Compute the mean and std of SM in the near environment

In [18]:
distance= 2;
######################## AMAZON ########################
smap_env_all_AB_2r_2015 = sel_composite(df_imerg_AB_2015_filt[(
   df_imerg_AB_2015_filt.longitude>-76.5)&(df_imerg_AB_2015_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_AB_2r_2016 = sel_composite(df_imerg_AB_2016_filt[(
   df_imerg_AB_2016_filt.longitude>-76.5)&(df_imerg_AB_2016_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_AB_2r_2017 = sel_composite(df_imerg_AB_2017_filt[(
   df_imerg_AB_2017_filt.longitude>-76.5)&(df_imerg_AB_2017_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_AB_2r_2018 = sel_composite(df_imerg_AB_2018_filt[(
   df_imerg_AB_2018_filt.longitude>-76.5)&(df_imerg_AB_2018_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_AB_2r_2019 = sel_composite(df_imerg_AB_2019_filt[(
   df_imerg_AB_2019_filt.longitude>-76.5)&(df_imerg_AB_2019_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_AB_2r_2020 = sel_composite(df_imerg_AB_2020_filt[(
   df_imerg_AB_2020_filt.longitude>-76.5)&(df_imerg_AB_2020_filt.longitude<-55.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')

######################## SESA ########################
smap_env_all_SESA_2r_2015 = sel_composite(df_imerg_SESA_2015_filt[(
    df_imerg_SESA_2015_filt.longitude>-76.5)&(df_imerg_SESA_2015_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_SESA_2r_2016 = sel_composite(df_imerg_SESA_2016_filt[(
    df_imerg_SESA_2016_filt.longitude>-76.5)&(df_imerg_SESA_2016_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_SESA_2r_2017 = sel_composite(df_imerg_SESA_2017_filt[(
    df_imerg_SESA_2017_filt.longitude>-76.5)&(df_imerg_SESA_2017_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_SESA_2r_2018 = sel_composite(df_imerg_SESA_2018_filt[(
    df_imerg_SESA_2018_filt.longitude>-76.5)&(df_imerg_SESA_2018_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_SESA_2r_2019 = sel_composite(df_imerg_SESA_2019_filt[(
    df_imerg_SESA_2019_filt.longitude>-76.5)&(df_imerg_SESA_2019_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')
smap_env_all_SESA_2r_2020 = sel_composite(df_imerg_SESA_2020_filt[(
    df_imerg_SESA_2020_filt.longitude>-76.5)&(df_imerg_SESA_2020_filt.latitude>-24.5)].reset_index(drop=True),
                               nmerged_smap,dist=distance,namevar='sm')

In [19]:
smap_env_all_AB_2r_2015.columns

Index(['time', 'local_time', 'feature', 'mean_SM_env', 'std_SM_env'], dtype='object')

In [20]:
print(len(smap_env_all_AB_2r_2015),len(df_imerg_AB_2015_filt[(
   df_imerg_AB_2015_filt.longitude>-76.5)&(df_imerg_AB_2015_filt.longitude<-55.5)]))

1803 1803


In [29]:
def add_SMvars_to_df(df1, df2, varname1='SM_std_env', varname2='SM_mean_env'):
    """
    Add columns from df2 to df1 based on matching 'feature' column values.
    
    df1: The dataframe where columns are added.
    df2: The dataframe whose columns are extracted.
    varname1 (str): The name of the mean column in df2 after renaming (default 'SM_mean_env_3hb').
    varname2 (str): The name of the std deviation column in df2 after renaming (default 'SM_std_env_3hb').

    Returns:
    DataFrame: The merged DataFrame with new columns added from df2.
    """
    # Copy df2 to avoid changing it outside of the function's scope
    df2_copy = df2.copy()
    
    # Rename columns to prevent conflicts during merge
    df2_copy.rename(columns={'std_SM_env': varname1, 'mean_SM_env': varname2}, inplace=True)
    
    # Check if 'feature' column exists in both dataframes
    if 'feature' not in df1.columns or 'feature' not in df2_copy.columns:
        raise KeyError("'feature' column must exist in both dataframes.")
    
    # Merge the dataframes on the 'feature' column with a left join
    merged_df = pd.merge(df1, df2_copy[['feature', varname1, varname2]], on='feature', how='left')
    
    return merged_df

In [32]:
######################## AMAZON ########################

df_2r_c_AB_2015 = add_SMvars_to_df(df_imerg_AB_2015_filt[(df_imerg_AB_2015_filt.longitude>-76.5)&(df_imerg_AB_2015_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2015.reset_index(drop=True));
df_2r_c_AB_2015 = add_SMvars_to_df(df_2r_c_AB_2015,smap_env_all_AB_2r_2015_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_AB_2016 = add_SMvars_to_df(df_imerg_AB_2016_filt[(df_imerg_AB_2016_filt.longitude>-76.5)&(df_imerg_AB_2016_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2016.reset_index(drop=True));
df_2r_c_AB_2016 = add_SMvars_to_df(df_2r_c_AB_2016,smap_env_all_AB_2r_2016_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_AB_2017 = add_SMvars_to_df(df_imerg_AB_2017_filt[(df_imerg_AB_2017_filt.longitude>-76.5)&(df_imerg_AB_2017_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2017.reset_index(drop=True));
df_2r_c_AB_2017 = add_SMvars_to_df(df_2r_c_AB_2017,smap_env_all_AB_2r_2017_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_AB_2018 = add_SMvars_to_df(df_imerg_AB_2018_filt[(df_imerg_AB_2018_filt.longitude>-76.5)&(df_imerg_AB_2018_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2018.reset_index(drop=True));
df_2r_c_AB_2018 = add_SMvars_to_df(df_2r_c_AB_2018,smap_env_all_AB_2r_2018_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_AB_2019 = add_SMvars_to_df(df_imerg_AB_2019_filt[(df_imerg_AB_2019_filt.longitude>-76.5)&(df_imerg_AB_2019_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2019.reset_index(drop=True));
df_2r_c_AB_2019 = add_SMvars_to_df(df_2r_c_AB_2019,smap_env_all_AB_2r_2019_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_AB_2020 = add_SMvars_to_df(df_imerg_AB_2020_filt[(df_imerg_AB_2020_filt.longitude>-76.5)&(df_imerg_AB_2020_filt.longitude<-55.5)],
                                   smap_env_all_AB_2r_2020.reset_index(drop=True));
df_2r_c_AB_2020 = add_SMvars_to_df(df_2r_c_AB_2020,smap_env_all_AB_2r_2020_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

In [33]:
######################## SESA ########################

df_2r_c_SESA_2015 = add_SMvars_to_df(df_imerg_SESA_2015_filt[(df_imerg_SESA_2015_filt.longitude>-76.5)&(df_imerg_SESA_2015_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2015.reset_index(drop=True));
df_2r_c_SESA_2015 = add_SMvars_to_df(df_2r_c_SESA_2015,smap_env_all_SESA_2r_2015_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_SESA_2016 = add_SMvars_to_df(df_imerg_SESA_2016_filt[(df_imerg_SESA_2016_filt.longitude>-76.5)&(df_imerg_SESA_2016_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2016.reset_index(drop=True));
df_2r_c_SESA_2016 = add_SMvars_to_df(df_2r_c_SESA_2016,smap_env_all_SESA_2r_2016_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_SESA_2017 = add_SMvars_to_df(df_imerg_SESA_2017_filt[(df_imerg_SESA_2017_filt.longitude>-76.5)&(df_imerg_SESA_2017_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2017.reset_index(drop=True));
df_2r_c_SESA_2017 = add_SMvars_to_df(df_2r_c_SESA_2017,smap_env_all_SESA_2r_2017_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_SESA_2018 = add_SMvars_to_df(df_imerg_SESA_2018_filt[(df_imerg_SESA_2018_filt.longitude>-76.5)&(df_imerg_SESA_2018_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2018.reset_index(drop=True));
df_2r_c_SESA_2018 = add_SMvars_to_df(df_2r_c_SESA_2018,smap_env_all_SESA_2r_2018_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_SESA_2019 = add_SMvars_to_df(df_imerg_SESA_2019_filt[(df_imerg_SESA_2019_filt.longitude>-76.5)&(df_imerg_SESA_2019_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2019.reset_index(drop=True));
df_2r_c_SESA_2019 = add_SMvars_to_df(df_2r_c_SESA_2019,smap_env_all_SESA_2r_2019_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

df_2r_c_SESA_2020 = add_SMvars_to_df(df_imerg_SESA_2020_filt[(df_imerg_SESA_2020_filt.longitude>-76.5)&(df_imerg_SESA_2020_filt.latitude>-24.5)],
                                   smap_env_all_SESA_2r_2020.reset_index(drop=True));
df_2r_c_SESA_2020 = add_SMvars_to_df(df_2r_c_SESA_2020,smap_env_all_SESA_2r_2020_6hb.reset_index(drop=True),
                                   varname1='SM_std_env_6hb',varname2='SM_mean_env_6hb');

In [34]:
df_2r_c_AB_2015.columns

Index(['index', 'feature', 'time', 'idx', 'area', 'mean', 'tot_prec',
       'max_prec', 'hdim_1', 'hdim_2', 'longitude', 'latitude', 'time_zone',
       'local_time', 'mean_ob', 'area_ob', 'max_ob', 'radius', 'frame', 'cell',
       'time_cell', 'stage', 'stage_n', 'time_diff_hours', 'delta_lat',
       'delta_lon', 'direction', 'cardinal_direction', 'categorized_direction',
       'topo_mean', 'topo_median', 'topo_max', 'topo_std', 'SM_std_env',
       'SM_mean_env', 'SM_std_env_6hb', 'SM_mean_env_6hb'],
      dtype='object')

In [37]:
df_2r_c_AB_2015.to_pickle('imerg_pkl/df_2r_c_AB_2015.pkl')
df_2r_c_AB_2016.to_pickle('imerg_pkl/df_2r_c_AB_2016.pkl')
df_2r_c_AB_2017.to_pickle('imerg_pkl/df_2r_c_AB_2017.pkl')
df_2r_c_AB_2018.to_pickle('imerg_pkl/df_2r_c_AB_2018.pkl')
df_2r_c_AB_2019.to_pickle('imerg_pkl/df_2r_c_AB_2019.pkl')
df_2r_c_AB_2020.to_pickle('imerg_pkl/df_2r_c_AB_2020.pkl')

df_2r_c_SESA_2015.to_pickle('imerg_pkl/df_2r_c_SESA_2015.pkl')
df_2r_c_SESA_2016.to_pickle('imerg_pkl/df_2r_c_SESA_2016.pkl')
df_2r_c_SESA_2017.to_pickle('imerg_pkl/df_2r_c_SESA_2017.pkl')
df_2r_c_SESA_2018.to_pickle('imerg_pkl/df_2r_c_SESA_2018.pkl')
df_2r_c_SESA_2019.to_pickle('imerg_pkl/df_2r_c_SESA_2019.pkl')
df_2r_c_SESA_2020.to_pickle('imerg_pkl/df_2r_c_SESA_2020.pkl')